<h1 style="padding-top: 25px;padding-bottom: 25px;text-align: left; padding-left: 10px; background-color: #DDDDDD; 
    color: black;"> <img style="float: left; padding-right: 10px; width: 45px" src="https://raw.githubusercontent.com/Harvard-IACS/2018-CS109A/master/content/styles/iacs.png"> CS109A Introduction to Data Science </h1>

## Homework 1: Data Collection, Parsing, and Quick Analyses

**Harvard University**<br/>
**Fall 2021**<br/>
**Instructors**: Pavlos Protopapas and Natesh Pillai<br/>
<hr style='height:2px'>

In [2]:
## RUN THIS CELL TO GET THE RIGHT FORMATTING 
import requests
from IPython.core.display import HTML
styles = requests.get("https://raw.githubusercontent.com/Harvard-IACS/2021-CS109A/master/themes/static/css/cs109.css").text
HTML(styles)


## Overview 

In this homework, your goal is to learn how to acquire, parse, clean, and analyze data. Toward this goal, we will address certain questions about COVID, and you will scrape data directly from a website. For the remainder of the semester, we will provide you data files directly; however, since real-world problems often require gathering information from a variety of sources, including the Internet, web scraping is a highly useful skill to have.

### Instructions
- To submit your assignment, follow the instructions given in Canvas.

### Learning Objectives
- Get started using [Jupyter Notebooks](https://jupyter.org/), which are incredibly popular, powerful, and will be our medium of programming for the duration of CS109A and CS109B.
- Become familiar with how to access and use data from various sources (i.e., web scraping and directly from files).
- Gain experience with data exploration and simple analysis.
- Become comfortable with [pandas](https://pandas.pydata.org/) as a means of storing and working with data.
- Reflect on what further analysis you may wish to do with this data. For example, given the material we've covered so far, what *more* do you wish you had the ability to do (e.g., modelling, prediction, etc). That is, think about questions you may have about the data, and try to imagine what types of tools you might need to help answer your questions.

### Notes
- Exercise **responsible scraping**. Web servers can become slow or unresponsive if they receive too many requests from the same source in a short amount of time. In your code, use a delay of 2 seconds between requests. This helps to not get blocked by the target website -- imagine how frustrating it would be to have this occur. Section 1 of this homework involves saving the scraped web pages to your local machine. Thus, after completing Section 1, you do not need to re-scrape any of the pages, unless you wish to occasionally grab the latest data. 

- <span style='color:purple'>**Web scraping requests can take several minutes**</span>. This is another reason why you should not wait until the last minute to do this homework.
- As you run a Jupyter Notebook, it maintains a running state of memory. Thus, <span style='color:purple'>the order in which you run cells matters</span> and plays a crucial role; it can be easy to make mistakes based on *when* you run different cells as you develop and test your code. Before submitting every Jupyter Notebook homework assignment, be sure to restart your Jupyter Notebook and run the entire notebook from scratch, all at once (i.e., "Kernel -> Restart & Run All"). Just make sure to not re-run the time intensive tasks unnecessarily. In this notebook for example, you could declare a variable to act as a 'setting' and use some controll logic to prevent a re-scrap from happening when not desired.

- We will be working with COVID data. COVID has impacted everyone in the world, and naturally some people have been greatly more affected than others. We, the teaching staff, are sensitive to this, empathize, and understand that working with COVID data may be unsettling to some. We apologize for any discomfort this may cause. Our intent with this assignment is purely pedagogical, and we'd like to remind students that data science and machine learning can be used to provide insights that can be used for good and invoke change. Toward this goal, parts of the homework are intended to shed light on the unfortunate, widespread inequality that exists. So, while this data may be unsettling, our aim is for the learned skills addressed here -- and in all future assignments -- to provide you with knowledge and confidence to do good work.

## 1. Obtaining Data (17 points)

For any given situation or scenario that we wish to understand, we will rely on having relevant data. Here, we are interested in the degree to which the SARS-CoV-2 virus has affected United States citizens (SARS-CoV-2 is the virus that causes the COVID-19 disease). The Centers for Disease Control and Prevention (CDC) provides relevant data from USAFacts.org that includes the number of confirmed COVID-19 cases on a per-county basis. Visit https://usafacts.org/visualizations/coronavirus-covid-19-spread-map/. At the bottom of the web page, in a blue table, you should see a list of every state, each of which has its own web page.

In this exercise, we will focus on automating the downloading of each state's data with [Requests](https://docs.python-requests.org/en/master/) and then manipulating it with [BeautifulSoup](https://www.crummy.com/software/BeautifulSoup/bs4/doc/). 

But first, as we will do for every Jupyter Notebook, let's import necessary packages that we will use throughout the notebook (i.e., run the cell below). 

In [3]:
# import the necessary libraries
import re
import requests
import pandas as pd
import numpy as np
from time import sleep
from bs4 import BeautifulSoup
import pickle # for loading a dictionary from disk
from typing import Optional # typehint that value can also be None

# NOTE: files will be saved to this directory, so you need to ensure
# that it exists on your system first (it should be visible from the
# directory of where you are running this Notebook file)
# i.e.,
# >> ls
# cs109a_hw1_student.ipynb
# data/
# state_data/
state_dir = "state_data/"

In [4]:
# we define this for convenience, as every state's url begins with this prefix
base_url = 'https://usafacts.org/visualizations/coronavirus-covid-19-spread-map/'

<div class='exercise'><b> Exercise 1.1 [1 pt]: Fetching Website data via Requests</b>

Fetch the web page located at `base_url` and save the request's returned object (a Response object) to a variable named `home_page`.
</div>

In [5]:
# YOUR CODE HERE
home_page = requests.get(base_url)
# END OF YOUR CODE HERE

<div class='exercise'><b>Exercise 1.2 [2 pts]:</b> In the cell below:
    
- Write a line of code that prints to the screen the status of `home_page` (the web page's returned object). You should receive a code of 200 if the request was successful; then,

- **When working with Jupyter Notebooks, avoiding unnecessarily long output in is essential.** Write code that prints the first 10,000 characters from the contents of `home_page` and [enable scolling output for the cell](https://www.youtube.com/watch?v=U4usAUZCv_c&t=1s).</div>


In [6]:
# YOUR CODE HERE
home_page
# END OF YOUR CODE HERE

<Response [200]>

<div class='exercise'><b> Exercise 1.3 [1 pt]:</b>
    
In the cell below, create a new BeautifulSoup object that parses the `home_page` as an HTML document (can be done with 1 line of code)</div>

In [7]:
# YOUR CODE HERE
home_page = BeautifulSoup(home_page.text, 'html.parser')
# END OF YOUR CODE HERE

<div class='exercise'><b> Exercise 1.4 [8 pts]:</b>
    
In the cell below, write code that uses the BeautifulSoup object to parse through the home page in order to extract the link for every state. Feel free to use [Regular Expressions]('https://docs.python.org/3/library/re.html'), in conjunction with any BeautifulSoup parsing. Specifically, the goal is to populate a `state_urls` [dictionary]('https://docs.python.org/3/tutorial/datastructures.html#dictionaries') by setting each key to be the state name and the value to be the full URL. When complete, there will be 51 keys (50 states + 1 for DC).

### AS A CRITICAL EXAMPLE:
Within `state_urls`, one of your <key, value> pairs should be:

``"District of Columbia" : "https://usafacts.org/visualizations/coronavirus-covid-19-spread-map/state/district-of-columbia"``

The casing here is **incredibly** important because later, in Exercise 4, you will merge your data with another dataset that has casing of this form. Thus, our key here should be `District of Columbia` and not `District Of Columbia` or `district-of-columbia`.


**NOTES:**
- There are _many_ solutions, but you may find it easiest to use Regular Expression(s)
- Pay attention to the casing example above, so that your later exercises go smoothly.
- Some HTML tag attributes may change over time. It your code stops working, make sure you are not targeting such ephemeral elements ('jss' class attributes are a common culprit)
</div>

In [8]:
state_urls = {}

# YOUR CODE HERE
list_state_name = [i.text for i in home_page.select("tbody a")]
state_url = ["https://usafacts.org"+i.get("href") for i in home_page.select("tbody a")]

for i,d in enumerate(list_state_name):
    state_urls[d] = state_url[i]
# END OF YOUR CODE HERE

Run the cell below to help ensure your formatting is correct and has 51 <key, value> pairs.

In [9]:
# SANITY CHECK
if len(state_urls.keys()) != 51 or \
state_urls["District of Columbia"] != "https://usafacts.org/visualizations/coronavirus-covid-19-spread-map/state/district-of-columbia":
    print("** 1.4 is incorrect")
else:
    print("** 1.4 might be correct")

** 1.4 might be correct


We wish to use the data without having to re-download it every time. So, let's save each webpage to our local hard drive. **NOTE: It's probably okay to download all of the state web pages a few times a day, but it's safer to keep it to a minimum.**

<div class='exercise'><b> Exercise 1.5 [5 pts]:</b>
    
In the cell below, we will iterate through all <key, value> items in `state_urls`. Your job is to make a web request for each URL and save the **contents** out to a file on your hard drive (use `state_dir`, defined above, as the prefix to the path.) 

**NOTES:**
- **Leave a 2 second pause between requests**
- You should be saving to a file the actual content of the webpage, not a BeautifulSoup object. That is, you should be able to open the saved files in an editor and see the HTML code, just as you could if you were to view the webpage in your browser and click 'View Page Source'.
- See [official Python documentation](https://docs.python.org/3/tutorial/inputoutput.html#reading-and-writing-files) for details on how to read/write files to disk
- You should have saved 51 different files to your hard drive.
- **Once you have written the files you can comment out this cell. This will save time and prevent you from making unnecessary requests when you restart the kernel & re-run all cells in the noteboook before submitting (as you should!)**
</div>

In [10]:
# 1.5 (5 pts) -- save each webpage to disk
for state, url in state_urls.items():
    
    # YOUR CODE HERE
    r = requests.get(url)
    with open(state_dir+state+".txt", 'w') as file:
        file.write(r.text)
    # END OF YOUR CODE HERE
    
    sleep(2) # LEAVE THIS IN

## 2. Loading and Exploring Data (22 pts)
Now, let's actually use the data! Fortunately, it's saved to our local machine, so we don't need to re-crawl the data every time we wish to access it. We want you to understand that [pandas](https://pandas.pydata.org/) is a library of useful data structures and operations, but we also wish to remind you that it isn't magic and it isn't the _only_ way to do Data Science; it's just a tool to help, and you could do the same operations without pandas. Thus, here we ask you to perform a few operations without using pandas, and then in Exercise 3 we will use pandas.

**Terminology Notice:** In the United States, every state is comprised of many **counties.** You can think of a **county** as being a pretty large district. 

First, run the cell below to construct `state_info`
This is an example of a Python [list comprehension](https://docs.python.org/3/tutorial/datastructures.html#list-comprehensions).

In [11]:
state_info = [(state, state_dir + state) for state in state_urls.keys()]

<div class='exercise'><b> Exercise 2.1 [10 pts]: Parsing and storing data</b>
    
Complete the `load_covid_data()` function, which:

- Takes as input `state_info`, which is a list of [tuples](https://docs.python.org/3.3/library/stdtypes.html?highlight=tuple#tuple): (state name, path to the corresponding file)
- Parses the contents of the file and extracts for **each county**:
    - 7 day average case
    - 7 day average deaths
    - \# of confirmed cases (total)
    - \# of deaths
    - Stores the above 4 pieces of data above as well as **population** in a **non-pandas** data structure named `covid_data` **for every county across every state**
- Returns `covid_data`
    <font color='blue'>


**NOTES:**
- **Attention: the population variable not in `state_info`. More on info on where to get this value is found in the green block below**
- To be clear, as of September 7, 2021, the webpage for Alabama currently lists 67 counties. District of Columbia has 1 county, and Wyoming has 23. Here we are asking you to store in `covid_data` *all counties* across every state. So, later, if we were wished to access just Wyoming's information, you could easily retrieve such for each of its 23 counties, or the info for any of the 67 counties in Alabama.
- `covid_data` **must not be a PANDAS data structure;** it must use a combination of lists and/or dictionaries. It's up to you to decide how to organize this, e.g., a lists of lists of lists, or a list of dictionaries, or a dictionary of dictionaries, or a dictionary of lists of lists, etc. A guiding decision should be ease of access for computing basic stats (Exercises 2.2, 2.3, and 2.4)
- For the duration of our using this data for the homework, be sure to **properly store the data with the correct data types;** that is, counts should be represented as Integers and rates should be represented as Floats. For example:
    - \# of confirmed cases (total) should be an **Integer**
    - \# of deaths should be an **Integer**
    - \# of confirmed cases (per 100k) should be a **Float** (we haven't created this feature yet!)
    - 7 day average cases should be an **Integer** (you'd think an average should be a float but the values you scrapped were rounded to the nearest int)
</div>

<div style='background-color:green;padding:15px'>
    <strong>Injecting population data</strong>
    

The table on usafacts.org you've just scrapped originally had additional columns related to county population. But these have recently been removed! We'd like you to be able to utilize the population data in the following section but also use up-to-date COVID data (so the [Internet Archive](https://archive.org/) was not an option). And, though this information is available elsewhere on usafacts.org, we've decided that you've already done enough web scraping for one HW. So below we've provided a [kludge](https://en.wikipedia.org/wiki/Kludge#Computer_science).
    
`population_dict` is a nested dictionary. The keys are states whose values are _themselves_ dictionaries. Those '_inner_' dictionaries' keys are counties and their values are populations. It looks like this:
```python
{'Alabama': {'Autauga County': 55869,
             'Baldwin County': 223234,
           ...
'Wyoming': {'Albany County': 38880,
            'Big Horn County': 11790,
            ...
```

To get at a population you could use double dictionary indexing like `population_dict['Alabama']['Autauga County']`

But not all of the counties you've scrapped have population data in this dictionary. So we've provided a helper function, `get_pop`, that will return `None` if the county data was not found. Use `get_pop` to inject popoulation data into your `covid_data` as you build it up in the `load_covid_data` function you'll implement below.
    
**Final Note: you should _ignore counties with missing population data or populations of 0_. Simply do not add them to `covid_data` as it is constructed.**
</div>

In [12]:
# load additional county population data as a nested dictionary
# you can read about this strange .pkl 'pickle' file here
# https://docs.python.org/3/library/pickle.html
with open('population.pkl', 'rb') as f:
    population_dict = pickle.load(f)

# not sure what's happening with the data types in the function header?
# check out: https://docs.python.org/3/library/typing.html#module-typing
def get_pop(state: str, county: str) -> Optional[int]:
    '''
    returns population of country, state (int)
    If county or state not found, returns None
    Example: get_pop('Alabama', 'Autauga County')
    '''
    try:
        return population_dict.get(state).get(county)
    except AttributeError:
        print('incorrect state name!')
        return None

In [13]:

# # # # 7 day average case
# # # # 7 day average deaths
# # # # # of confirmed cases (total)
# # # # # of deaths
# # # # Stores the above 4 pieces of data above as well as population in a non-pandas data structure named covid_data for every county across every state

# def load_covid_data(state_info):
#     covid_data = {}
#     # YOUR CODE HERE
#     for state in state_info:
#         with open(state[1]+".txt", 'r') as file:
#             myfile = file.read()
#             myfile = BeautifulSoup(myfile, 'html.parser')
#             numbers = myfile.select('tbody tr td')
            
#             covid_data[state[0]] = {} # state
#             ind = 6
#             for i in myfile.select('tbody tr a'):

#                 covid_data[state[0]][i.text] = {} # county
#                 covid_data[state[0]][i.text]["ave_case"] = int((numbers[ind].text).replace(",","")) # 7 day average case
#                 covid_data[state[0]][i.text]["ave_death"] = int((numbers[ind+1].text).replace(",","")) # 7 day average deaths
#                 covid_data[state[0]][i.text]["total_case"] = int((numbers[ind+2].text).replace(",","")) # # of confirmed cases (total)
#                 covid_data[state[0]][i.text]["total_death"] = int((numbers[ind+3].text).replace(",","")) # # of deaths
#                 if (get_pop(state[0],i.text) != 0 and get_pop(state[0],i.text) != None):
#                     covid_data[state[0]][i.text]["population"] = get_pop(state[0],i.text)
#                 ind = ind+4
#     # END OF YOUR CODE HERE
#     return covid_data


# ###
# # covid_data = []

# # for state in state_info:
    
# #     with open(state[1]+".txt", 'r') as file:
# #         myfile = file.read()
# #         myfile = BeautifulSoup(myfile, 'html.parser')
# #         numbers = myfile.select('tbody tr td')
        
# #         ind = 6

# #         for i in myfile.select('tbody tr a'):
# #             covid_data_state = {}
# #             covid_data_state["state"] = state[0] # state
# #             covid_data_state["county"] = i.text  # county
# #             covid_data_state["ave_case"] = int((numbers[ind].text).replace(",","")) # 7 day average case
# #             covid_data_state["ave_death"] = int((numbers[ind+1].text).replace(",","")) # 7 day average deaths
# #             covid_data_state["total_case"] = int((numbers[ind+2].text).replace(",","")) # # of confirmed cases (total)
# #             covid_data_state["total_death"] = int((numbers[ind+3].text).replace(",","")) # # of deaths
# #             if (get_pop(state[0],i.text) != 0 and get_pop(state[0],i.text) != None):
# #                 covid_data_state["population"] = get_pop(state[0],i.text)
# #             ind = ind+4
# #             covid_data.append(covid_data_state)
# # covid_data

    

In [14]:
def load_covid_data(state_info):
#     covid_data = {}
    # YOUR CODE HERE
    covid_data = []

    for state in state_info:

        with open(state[1]+".txt", 'r') as file:
            myfile = file.read()
            myfile = BeautifulSoup(myfile, 'html.parser')
            numbers = myfile.select('tbody tr td')

            ind = 6

            for i in myfile.select('tbody tr a'):
                county = re.sub(r"\sof$", "", (i.text).replace(",","").replace("–","-"))
                covid_data_state = {}
                covid_data_state["state"] = state[0] # state
                covid_data_state["county"] = county  # county
                covid_data_state["ave_case"] = int((numbers[ind].text).replace(",","")) # 7 day average case
                covid_data_state["ave_death"] = int((numbers[ind+1].text).replace(",","")) # 7 day average deaths
                covid_data_state["total_case"] = int((numbers[ind+2].text).replace(",","")) # # of confirmed cases (total)
                covid_data_state["total_death"] = int((numbers[ind+3].text).replace(",","")) # # of deaths
                if (get_pop(state[0],county) != 0 and get_pop(state[0],county) != None):
                    covid_data_state["population"] = int(get_pop(state[0],county))
                ind = ind+4
                covid_data.append(covid_data_state)
    # END OF YOUR CODE HERE
    return covid_data

Run the cell below (no changes necessary) to execute your code above

In [15]:
covid_data = load_covid_data(state_info)

In [16]:
population_dict

{'Alabama': {'Autauga County': 55869,
  'Baldwin County': 223234,
  'Barbour County': 24686,
  'Bibb County': 22394,
  'Blount County': 57826,
  'Bullock County': 10101,
  'Butler County': 19448,
  'Calhoun County': 113605,
  'Chambers County': 33254,
  'Cherokee County': 26196,
  'Chilton County': 44428,
  'Choctaw County': 12589,
  'Clarke County': 23622,
  'Clay County': 13235,
  'Cleburne County': 14910,
  'Coffee County': 52342,
  'Colbert County': 55241,
  'Conecuh County': 12067,
  'Coosa County': 10663,
  'Covington County': 37049,
  'Crenshaw County': 13772,
  'Cullman County': 83768,
  'Dale County': 49172,
  'Dallas County': 37196,
  'DeKalb County': 71513,
  'Elmore County': 81209,
  'Escambia County': 36633,
  'Etowah County': 102268,
  'Fayette County': 16302,
  'Franklin County': 31362,
  'Geneva County': 26271,
  'Greene County': 8111,
  'Hale County': 14651,
  'Henry County': 17205,
  'Houston County': 105882,
  'Jackson County': 51626,
  'Jefferson County': 658573,
  

<div class='exercise'><b> Exercise 2.2 [4 pts]: Simple analytics</b>
    
Complete the `calculate_county_stats()` function, which calculates:
1. The single county (and the state to which it belongs) that has the **lowest rate** of COVID cases per 100k people
2. The single county (and the state to which it belongs) that has the **highest rate** of COVID cases per 100k people
   
**NOTES:**
- Place your resulting variables within the blanks of the `print()` statements that we provide
- These values you report should be Floating point numbers (e.g., 3.4), not Integers (e.g., 3).
- If there are ties, return any one of the tied counties (see if you can do it in an unbiased way!)
</div>

In [51]:
# QUESTION: what is the output? Any variable should be returned? what do you mean by "ties"?

def calculate_county_stats(covid_data):
    
    # YOUR CODE HERE
    tmp = pd.DataFrame(covid_data)
    tmp['total_case'] = tmp['total_case'] / 100000
    tmp_min = tmp[tmp["total_case"] == min(tmp["total_case"])]
    tmp_max = tmp[tmp["total_case"] == max(tmp["total_case"])]
    states_min = ", ".join([i for i in tmp_min["state"]])
    states_max = ", ".join([i for i in tmp_max["state"]])
    counties_min = ", ".join([i for i in tmp_min["county"]])
    counties_max = ", ".join([i for i in tmp_max["county"]])
    
    print(counties_min + " has the lowest COVID cases per 100k: " + str(round(tmp_min["total_case"].iloc[0],1)))
    print(counties_max + " has the highest COVID cases per 100k: " + str(round(tmp_max["total_case"].iloc[0],1)))
    # END OF YOUR CODE HERE
    

Run the cell below (no changes necessary) to execute your code above

In [52]:
calculate_county_stats(covid_data)

Hoonah-Angoon Census Area, Lake and Peninsula Borough, Yakutat City and Borough, Kalawao County has the lowest COVID cases per 100k: 0.0
Los Angeles County has the highest COVID cases per 100k: 13.6


<div class='exercise'><b> Exercise 2.3 [4 pts]: Simple analytics</b>
    
Complete the `calculate_state_deaths()` function, which calculates:
1. The state that has the **lowest number** of deaths
2. The state that has the **highest number** of deaths

**NOTES:**
- Place your resulting variables within the blanks of the `print()` statements that we provide (don't just manually type your textual answers in the blanks)
- These values you report should be Integers, not Floating point numbers.
- If there are ties, return any of the tied states

</div>

In [53]:
def calculate_state_deaths(covid_data):
    
    # YOUR CODE HERE
    tmp = pd.DataFrame(covid_data)
    frame = tmp.groupby("state")["total_death"].agg("sum").to_frame().sort_values(by = "total_death").reset_index()
    print(frame.iloc[0][0] + " has the fewest COVID deaths: " + str(frame.iloc[0][1]))
    print(frame.iloc[-1][0] + " has the most COVID deaths: " + str(frame.iloc[-1][1]))
    # END OF YOUR CODE HERE


Run the cell below (no changes necessary) to execute your code above

In [54]:
calculate_state_deaths(covid_data)

Vermont has the fewest COVID deaths: 287
California has the most COVID deaths: 66232


<div class='exercise'><b> Exercise 2.4 [4 pts]: Simple analytics</b>
    
Complete the `calculate_state_deathrate()` function, which calculates:
1. The state that has the **lowest rate** of deaths based on its entire population
2. The state that has the **highest rate** of deaths based on its entire population

**NOTES:**
- To calculate a state's population, we are asserting that is sufficient to sum the population over all counties, and that each county's population can be calculated simply from the data fields stored within `covid_data`.
- **If a county has reported 0 COVID cases,** then we should ignore this county as we estimate its county population. Thus, that county would contribute 0 to its state population total.
- Round your results to the a single person (e.g., "1 out of every 2703 people has died" not 2703.4)
- Place your resulting variables within the blanks of the `print()` statements that we provide (don't just manually type your textual answers in the blanks)
</div>

In [60]:
def calculate_state_deathrate(covid_data):
    
    # YOUR CODE HERE
    tmp = pd.DataFrame(covid_data)
    frame = tmp.groupby("state")[["total_death", "population"]].agg("sum").reset_index()
    frame["death_rate"] = frame["population"] /frame["total_death"]
    frame1 = frame.sort_values(by = "death_rate").reset_index()
    print(frame1.iloc[-1]["state"] + " has the lowest COVID death rate; 1 out of every " + str(round(frame1.iloc[-1]["death_rate"])) + " people has died")
    print(frame1.iloc[0]["state"] + " has the highest COVID death rate; 1 out of every " + str(round(frame1.iloc[0]["death_rate"])) + " people has died")
    # END OF YOUR CODE HERE
    

Run the cell below (no changes necessary) to execute your code above

In [61]:
calculate_state_deathrate(covid_data)

Vermont has the lowest COVID death rate; 1 out of every 2174 people has died
New Jersey has the highest COVID death rate; 1 out of every 334 people has died


## 3. PANDAS (36 pts)
What if we wanted to observe more than just the single-most extreme counties and states? What if we wanted to inspect all states, after having sorted the data by some feature? As you saw in the above exercises, doing the most basic analytics is possible, but it can quickly become cumbersome. As we learned in class, PANDAS is a great library that provides data structures that are highly useful for data analysis.

<div class='exercise'><b> Exercise 3.1 [10 pts]: Converting to PANDAS</b>

In Exercise 2, we worked with `covid_data`, which is comprises of some combination of lists and/or dictionaries.

Complete the `convert_to_pandas()` function, which converts `covid_data` to a PANDAS DataFrame, whereby:
- Each row corresponds to a unique county
- The 4 columns are:
    - county
    - state
    - \# total covid cases (Integer)
    - \# case per 100k (Integer)
    - \# covid deaths (Integer)
- The columns should be titled **exactly** as listed above

**NOTE:**
- If there exists multiple counties with the same name, each of which belonging to a different state, then there should be a distinct row for each.
- The 2 columns that correspond to COVID counts should all be Integers (e.g., 1498), not Floating point digits (e.g., 1498.0)
</div>

In [23]:
def convert_to_pandas(covid_data):
    
    # YOUR CODE HERE
    df = pd.DataFrame(covid_data)
    df["# case per 100k"] = df["total_case"] / 100000
    df = df.rename(columns = {"total_case":"# total covid cases", "total_death":"# covid deaths"})
    covid_df = df[["county","state", "# total covid cases", "# case per 100k", "# covid deaths"]]

    # END OF YOUR CODE HERE
    return covid_df


Run the cell below (no changes necessary) to execute your code above and inspect the results.

In [24]:
covid_df = convert_to_pandas(covid_data)

In [25]:
covid_df.head()

county    state  # total covid cases  # case per 100k  \
0  Autauga County  Alabama                 9207          0.09207   
1  Baldwin County  Alabama                34726          0.34726   
2  Barbour County  Alabama                 3260          0.03260   
3     Bibb County  Alabama                 3752          0.03752   
4   Blount County  Alabama                 9095          0.09095   

   # covid deaths  
0             119  
1             411  
2              67  
3              74  
4             146

In [26]:
covid_df.shape

(3142, 5)

<div class='exercise'><b> Exercise 3.2 [5 pts]: Simple analytics</b>

Complete the `calculate_county_stats2()` function, **which should obtain identical information (other than ties) as problem 2.2, but now using the PANDAS `covid_df` DataFrame.**

That is, it should calculates:
1. the single county (and the state to which it belongs) that has the **lowest rate** of COVID cases per 100k people
2. the single county (and the state to which it belongs) that has the **highest rate** of COVID cases per 100k people

**NOTES:**
- If there are ties, return any of the tied counties
- Place your resulting variables within the `print()` statements that we provide (don't just manually type your textual answers in the blanks)
- The values you report should be Floating point numbers (e.g., 3.4), not Integers (e.g., 3).

</div>

In [27]:
def calculate_county_stats2(covid_df):

    # YOUR CODE HERE
    tmp_min = covid_df[covid_df["# case per 100k"] == min(covid_df["# case per 100k"])]
    tmp_max = covid_df[covid_df["# case per 100k"] == max(covid_df["# case per 100k"])]
    states_min = ", ".join([i for i in tmp_min["state"]])
    states_max = ", ".join([i for i in tmp_max["state"]])
    counties_min = ", ".join([i for i in tmp_min["county"]])
    counties_max = ", ".join([i for i in tmp_max["county"]])
    print("{} from {} respectively have the lowest rate of COVID cases per 100k people: {:.1f}".format(counties_min,states_min,tmp_min["# case per 100k"].iloc[0]))
    print("{} from {} has the higher rate of COVID cases per 100k people: {:.1f}".format(counties_max,states_max,tmp_max["# case per 100k"].iloc[0]))

    # END OF YOUR CODE HERE

Run the cell below (no changes necessary) to execute your code above

In [28]:
calculate_county_stats2(covid_df)

Hoonah-Angoon Census Area, Lake and Peninsula Borough, Yakutat City and Borough, Kalawao County from Alaska, Alaska, Alaska, Hawaii respectively have the lowest rate of COVID cases per 100k people: 0.0
Los Angeles County from California has the higher rate of COVID cases per 100k people: 13.6


<div class='exercise'><b> Exercise 3.3 [5 pts]: Simple analytics</b>
    
Complete the `calculate_state_deaths2()` function, **which should obtain identical information as problem 2.3 (other than ties), but now using the PANDAS `covid_df` DataFrame.**
1. the state that has the **lowest number** of deaths
2. the state that has the **highest number** of deaths

**NOTES:**
- If there are ties, return any of the tied states
- Place your resulting variables within the `print()` statements that we provide (don't just manually type your textual answers in the blanks)
- The values you report should be Integers, not Floating point numbers.
</div>

In [29]:
def calculate_state_deaths2(covid_df):
    
    # YOUR CODE HERE
    frame = covid_df.groupby("state")["# covid deaths"].agg("sum").to_frame().sort_values(by = "# covid deaths").reset_index()
    print("{} has the lowest number of COVID deaths: {:d}".format(frame.iloc[0][0],frame.iloc[0][1]))
    print("{} has the higher number of COVID deaths: {:d}".format(frame.iloc[-1][0],frame.iloc[-1][1]))
    # END OF YOUR CODE HERE

Run the cell below (no changes necessary) to execute your code above

In [30]:
calculate_state_deaths2(covid_df)

Vermont has the lowest number of COVID deaths: 287
California has the higher number of COVID deaths: 66232


<div class='exercise'><b> Exercise 3.4 [5 pts]: Simple analytics</b>
    
Complete the `calculate_state_deathrate2()` function, **which should obtain identical information as problem 2.4, but now using the PANDAS `covid_df` DataFrame.** That is, return:

1. The state that has the **lowest rate** of deaths based on its entire population
2. The state that has the **highest rate** of deaths based on its entire population

**NOTES:**
- Just as in, 2.4, to calculate a state's population, we are asserting that is sufficient to sum the population over all counties -- and that each county's population can be calculated simply from the data fields stored within `covid_data`.
- Just as in 2.4, counties with 0 COVID cases should contibute 0 to the total population of the state.
- Round your results to the a single person (e.g., "1 out of every 2703 people has died" not 2703.4)
- Place your resulting variables within the blanks of the `print()` statements that we provide (don't just manually type your textual answers in the blanks)
</div>

In [31]:
def calculate_state_deathrate2(covid_df):
    
    # YOUR CODE HERE

    covid_df["population"] = pd.DataFrame(covid_data)["population"]
    frame = covid_df.groupby("state")[["# covid deaths", "population"]].agg("sum").reset_index()
    frame["death_rate"] = frame["population"] / frame["# covid deaths"]
    frame1 = frame.sort_values(by = "death_rate").reset_index()
#     print("{} has the lowest rate of COVID deaths: {:.0f}".format(frame1.iloc[0]["state"],frame1.iloc[0]["death_rate"]))
#     print("{} has the higher rate of COVID deaths: {:.0f}".format(frame1.iloc[-1]["state"],frame1.iloc[-1]["death_rate"]))

    # END OF YOUR CODE HERE
    print(str(frame1.iloc[-1]["state"]) + " has the lowest COVID death rate; 1 out of every " + str(round(frame1.iloc[-1]["death_rate"])) + " people has died")
    print(str(frame1.iloc[0]["state"]) + " has the highest COVID death rate; 1 out of every " + str(round(frame1.iloc[0]["death_rate"])) + " people has died")
    


Run the cell below (no changes necessary) to execute your code above

In [32]:
calculate_state_deathrate2(covid_df)

Vermont has the lowest COVID death rate; 1 out of every 2174 people has died
New Jersey has the highest COVID death rate; 1 out of every 334 people has died


These are highly alarming and tragic statistics, and doing calculations like this can really put the severity of the virus into a grounded perspective. In order to perfectly understand the virus and its spread, everyone would be tested and we would have contact tracing. Without getting into socio-political issues, our point is that (1) we wish to better understand the virus' effects; (2) naturally, any real-world data is messy, and thus we will never have _perfect_ data.


Let's now attempt to understand _some_ of the uncertainty around our COVID data. It's reasonable to believe that the # of COVID deaths is fairly reliable. That is, there are inevitably some false negatives -- people who died of COVID but were not accounted for, as other conditions were listed as the cause. However, the number of false positives is probably minimal -- if someone was denoted as dying from COVID, it's probably true. It's also the case that every disease has a mortality rate. For example, if 1,000 randomly-selected people contracted COVID, $N\%$ of them will die. We'd imagine that this percentage should be pretty constant throughout all people in the United States. Of course, we can think of reasons for this rate to not be perfectly consistent, as some people are at higher risk (e.g., older folks, people with pre-existing conditions, etc). Yet, we can imagine that this natural *variance* in the population to be fairly uniform throughout the USA at large. To this end, if all counties were equal in their **testing**, we ought to see a consistent ratio between: (a) the # of people who died from COVID; and (b) the # of people who tested positive for COVID. Within the medical domain, this ratio is referred to as the `case_fatality_rate`. For example, if 750 people tested positive for COVID, and 75 of those people died, then our `case_fatality_rate` would be 0.1 (meaning 10%).

<div class='exercise'><b>Exercise 3.5 [5 pts]: Further analytics</b>
    
Complete the `add_death_stats()` function below, which should add 3 new columns:
- `case_fatality_rate`
- `# covid deaths per 100k` and
- `population`

And return the updated DataFrame **sorted by `case_fatality_rate` in ascending order** 

**NOTES:**

- `add_death_stats()` should return a new DataFrame that has 8 columns:
    - county
    - state
    - population
    - \# total covid cases
    - \# covid cases per 100k
    - \# covid deaths
    - \# covid deaths per 100k
    - case_fatality_rate
- DataFrame should be sorted by `case_fatality_rate` in ascending order
- Again, the values for `case_fatality_rate` should be < 1. A value of 1 would mean that 100% of people who tested positive for COVID also died.
- `# covid deaths per 100k` is simply defined as the # of COVID deaths for every 100,000 people. We calculate this on a per-county basis.
- Make sure you inspect your results thoroughly. You may have to address the results of divisions by zero (or prevent these divisions in the first place). 
</div>

In [33]:
def add_death_stats(covid_df):
    
    # can add an infintesimal or fillna after the fact to handle nans from divide by 0.
    
    # YOUR CODE HERE
    covid_df["case_fatality_rate"] = covid_df["# covid deaths"]/covid_df["# total covid cases"]
    covid_df["# covid deaths per 100k"] = covid_df["# covid deaths"]/100000
    covid_df["population"] = pd.DataFrame(covid_data)["population"]
    covid_df = covid_df.sort_values(by = "case_fatality_rate")
    covid_df = covid_df.replace(np.nan, np.inf)
    # END OF YOUR CODE HERE
    return covid_df

Run the cell below (no changes necessary) to execute your code above

In [34]:
covid_updated = add_death_stats(covid_df)
covid_updated
covid_updated[["case_fatality_rate","# covid deaths per 100k"]][:-5].agg("mean")

case_fatality_rate         0.018157
# covid deaths per 100k    0.002038
dtype: float64

<div class='exercise'><b>Reflection:</b> Data Analysis allows us to better understand a system or scenario.
</div>

<div class='exercise'><b>Exercise 3.6.1 [2 pts] Trends</b>
    
Having looked at the results from Exercises 3.3, 3.4, and 3.5, what are some trends you've noticed and any conclusions you have? (2-3 sentences)?</div>

<div style='background-color:#F6FEFA;padding:15px'>

**Some counties (e.g. Harding County in New Mexico) have low number or rate of cases but high case fatality rate. While some counties (e.g. Rich County in Utah) have high number or rate of cases but low case fatality rate. This may imply the effectiveness of virus control actions and the performance of local healthcare system.**

</div>


<div class='exercise'><b>Exercise 3.6.2 [2 pts]: Data Reliability</b>
    
Having looked at the results from Exercise 3.5 (i.e., `covid_updated` DataFrame), do you think the original data is reliable and accurate? Are there any potential biases that you're aware of or concerned about? Please explain (3-5 sentences).</div>

<div style='background-color:#F6FEFA;padding:15px'>

**The case fatality rate is not constant throughout the country. I doubt the reliability and accuracy of the original data because there may be biases when reporting deaths -- people who died of COVID but were not accounted for. And it is likely that not all the people are tested, so some cases were not accounted for. And there may be variations in the testing availability and capability among the counties.**

</div>


<div class='exercise'><b>Exercise 3.6.3 [1 pt]: Relationships Between Variables</b>
    
If a county has 15 confirmed deaths, how many cases would you expect? What would you expect its population to be? Explain why (1-2 sentences in total)?

**NOTE:** For this question, we aren't evaluating the accuracy of your answer but your thought-process and reasoning.
</div>

<div style='background-color:#F6FEFA;padding:15px'>

**Since the mean case fatality rate is 0.018, if a county has 15 confirmed deaths, it is expected that there are 15/0.018 = 833 cases.**

</div>


<div class='exercise'><b>Exercise 3.6.4 [1 pt]: Further Questions</b>
    
What further questions do you wish to answer about COVID, including ones that may not be possible to answer from this data alone (e.g., Is there a correlation between the average age of people in a county and the # of COVID deaths)? Write at least 3 of your questions.</div>

<div style='background-color:#F6FEFA;padding:15px'>


**1. How vaccination rate influences the case rate and the case fatality rate?
2. Is keeping social distance effective on reduing the case rate?
3. Is there a correlation between the gender and the COVID deaths/cases?**

</div>


## 4. MORE DATA (25 pts)
In order to better understand how COVID (and the testing thereof) has impacted our world, we could look at how it relates to demographics, income, education, health, and political voting. For this exercise, we will make use of `election2020_by_county.csv`.

<div class='exercise'><b>Exercise 4.1 [4 pts]: Load more data</b>

Complete the `merge_data()` function, which should:
1. First, load `election2020_by_county.csv` as a new DataFrame.
2. Then, using the state and county names (case-sensitive) in both DataFrames, merge this new DataFrame with your existing `covid_updated`.
3. Return the merged DataFrame

The returned `merged` DataFrame should contain all 8 columns from `covid_updated`:
- county
- state
- \# total covid cases
- \# covid cases per 100k
- \# covid deaths
- population
- \# covid deaths per 100k
- case_fatality_rate

along with these 15 columns from `election2020_by_county.csv`:
- hispanic
- minority
- female
- unemployed
- income
- nodegree
- bachelor
- inactivity
- obesity
- density
- cancer
- voter_turnout
- voter_gap
- trump
- biden

**NOTES:**
- We are dropping two columns from `election2020_by_county.csv`:
    - fipscode
    - population
- Do not attempt to manually fix any of the state or county names. That is, **our merging should require the state and county names to be identical (case-sensitive) between the two DataFrames.** If there is a discrepancy between the two, do not worry about adjusting these names to find a perfect match.

**HINT:** there are many ways to solve this, but you may find the [pandas.merge()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.merge.html) function can be really helpful

**EXTRA INFORMATION:** In case you're wondering what the different features/columns are in `election2020_by_county.csv`:

- state: the state in which the county lies
- fipscode: an ID to identify each county
- county: the name of each county
- population: total population
- hispanic: percent of adults that are hispanic
- minority: percent of adults that are nonwhite
- female: percent of adults that are female
- unemployed: unemployment rate, as a percent
- income: median income
- nodegree: percent of adults who have not completed high school
- bachelor: percent of adults with a bachelor’s degree
- inactive: percent of adults who do not exercise in their leisure time
- obesity: percent of adults with BMI > 30
- density: population density, persons per square mile of land
- cancer: prevalence of cancer per 100,000 individuals
- voter_turnout: percentage of voting age population that voted
- voter_gap: percentage point gap in 2020 presidential voting: trump-briden
</div>

In [35]:
def merge_data(covid_updated, filepath):
    
    # YOUR CODE HERE
    election_data = pd.read_csv(filepath).drop(columns = ["population", "fipscode"])
    new_df = pd.merge(election_data, covid_updated,  how='inner', left_on=['state','county'], right_on = ['state','county'])
    # END OF YOUR CODE HERE
    return new_df

Run the cell below (no changes necessary) to execute your code above

In [36]:
merged = merge_data(covid_updated, 'election2020_by_county.csv')

In [37]:
merged.head()

state          county  hispanic  minority     female  unemployed  income  \
0  Alabama  Autauga County       2.7      24.6  51.124718         5.2   54366   
1  Alabama  Baldwin County       4.4      16.9  51.058714         5.5   49626   
2  Alabama  Barbour County       4.2      54.3  46.658524         8.9   34971   
3  Alabama     Bibb County       2.4      25.4  45.744021         6.6   39546   
4  Alabama   Blount County       9.0      12.6  50.595661         5.4   45567   

   nodegree  bachelor  inactivity  ...  voter_turnout  voter_gap  trump  \
0      13.8      21.9        28.6  ...      32.614102       44.5   71.5   
1      11.0      28.6        22.3  ...      29.882994       53.9   76.2   
2      25.4      13.6        31.8  ...      48.408900        8.0   53.6   
3      22.1      10.2        33.9  ...      45.798890       57.7   78.4   
4      21.9      12.3        28.0  ...      35.410345       80.0   89.6   

   biden  # total covid cases  # case per 100k  # covid deaths  population  \
0   27.0                 9207          0.09207             119     55869.0   
1   22.3                34726          0.34726             411    223234.0   
2   45.6                 3260          0.03260              67     24686.0   
3   20.7                 3752          0.03752              74     22394.0   
4    9.6                 9095          0.09095             146     57826.0   

   case_fatality_rate  # covid deaths per 100k  
0            0.012925                  0.00119  
1            0.011836                  0.00411  
2            0.020552                  0.00067  
3            0.019723                  0.00074  
4            0.016053                  0.00146  

[5 rows x 23 columns]

In [38]:
merged.shape

(3016, 23)

As mentioned above, the merging requires exact matching between the two DataFrames' `state` and `county` columns. Thus, some mismatches will occur, yielding our `merged` DataFrame to have fewer rows than `covid_updated` and `election2016_by_county.csv`.

<div class='exercise'><b>Data Construction / Understanding</b>
</div>

<div class='exercise'><b>Exercise 4.2.1 [1 pt]: Lost Rows</b>
    
Compared to `covid_updated`, how many rows were lost during this merging process to create `merged`? Running the cell below should print to the screen your answer.
</div>

In [39]:
# YOUR CODE HERE
covid_updated.shape[0]-merged.shape[0]
# END OF YOUR CODE HERE

126

<div class='exercise'><b>Exercise 4.2.2 [2 pts]: Lost Counties</b>  

List the county and state of *at least 3* such rows that exist in `covid_updated` but didn't make it into `merged`. Running the cell below should print to the screen your answer.
</div>

In [40]:
# YOUR CODE HERE
covid_updated[(~covid_updated.state.isin(merged.state))&(~covid_updated.county.isin(merged.county))].iloc[:3,:2]
# END OF YOUR CODE HERE

county      state
1124      Cameron Parish  Louisiana
1150  Plaquemines Parish  Louisiana
1158   St. Helena Parish  Louisiana

<div class='exercise'><b>Exercise 4.2.3 [2 pts]: Suggested Fixes</b>
   
If we needed to be highly thorough and needed comprehensive data coverage, do you have any suggestions on how we could quickly, soundly fix most or all of them? (Write 2-3 sentences.)
    
<b>NOTE: Please do not actually fix these mismatches; for this Exercise, it's okay that the `merged` DataFrame is smaller than `covid_updated`</b>
</div>

<div style='background-color:#F6FEFA;padding:15px'>

**To soundly fix most or all of them, I would check data state by state. I would check if the number of counties of each state matches between two DataFrame. If not, I would check carefully the names of the counties of that state and see why there is a mismatch, and then try to fix it. This approach may not be efficient or quick.**

</div>


This past example demonstrates how easy it is for data to become messy. It also shows the importance of paying close attention to your data in order to understand what you are working with.

Our `case_fatality_rate` column can be viewed as an approximation of how effective and thorough *COVID testing* is for a given county.

Our `# covid deaths` column can be viewed as an extreme indication of how severe *COVID* has impacted a given county.

Our `# covid cases per 100k` column be viewed as middle-ground between the two aforementioned features. That is, it measures the impact of the disease and is influenced by the thoroughness of COVID testing.

Using these three informative features, we can inspect how impacted each county is, while correlating this with other features of each county, such as income-level, health metrics, demographics, etc. 

<div class='exercise'><b>Exercise 4.3 [2 pts]: Cleaning the data</b>

Before we do any further analysis, we first notice that some counties haven't encountered a single COVID death (usually ones with very small populations), thus providing us with little information. Write code in the cell below to update the `merged` DataFrame so that all rows with 0 deaths are removed.

In [41]:
# YOUR CODE HERE
merged.drop(merged[merged["# covid deaths"] == 0].index, inplace = True)
# END OF YOUR CODE HERE

Running `.describe()` allows us to quickly see summary statistics of our DataFrame

In [42]:
merged.describe()

hispanic     minority       female   unemployed         income  \
count  2981.000000  2981.000000  2981.000000  2982.000000    2982.000000   
mean      9.265012    22.548105    49.917860     5.493561   47059.404427   
std      13.928841    19.823039     2.357761     1.968451   11941.876675   
min       0.000000     0.200000    19.166215     1.800000   21658.000000   
25%       2.000000     6.900000    49.463507     4.100000   38900.250000   
50%       4.000000    15.200000    50.383576     5.300000   45186.000000   
75%       9.500000    33.700000    51.068718     6.500000   52522.750000   
max      99.200000    99.400000    56.633907    24.000000  122641.000000   

          nodegree     bachelor   inactivity      obesity       density  ...  \
count  2982.000000  2982.000000  2982.000000  2982.000000   2982.000000  ...   
mean     14.976526    19.996009    25.964185    30.991818    230.269987  ...   
std       6.746133     8.730142     5.158233     4.450938   1711.636679  ...   
min       1.900000     4.400000     8.100000    11.800000      0.100000  ...   
25%       9.900000    14.000000    22.700000    28.400000     17.000000  ...   
50%      13.500000    17.900000    25.800000    31.200000     44.850000  ...   
75%      19.200000    23.600000    29.400000    33.800000    110.075000  ...   
max      53.300000    72.000000    41.400000    47.600000  69468.400000  ...   

       voter_turnout    voter_gap        trump        biden  \
count    2947.000000  2946.000000  2946.000000  2946.000000   
mean       35.514568    33.039477    65.658282    32.618805   
std        13.829715    30.882108    15.504336    15.387119   
min      -168.323353   -90.000000     4.000000     3.100000   
25%        27.701651    15.450000    56.825000    20.800000   
50%        35.049804    39.300000    68.800000    29.500000   
75%        42.476078    56.875000    77.575000    41.500000   
max       100.000000    93.100000    96.200000    94.000000   

       # total covid cases  # case per 100k  # covid deaths    population  \
count         2.982000e+03      2982.000000     2982.000000  2.982000e+03   
mean          1.291457e+04         0.129146      207.049631  1.061675e+05   
std           4.405656e+04         0.440566      778.060083  3.404455e+05   
min           1.400000e+01         0.000140        1.000000  4.040000e+02   
25%           1.366250e+03         0.013663       23.000000  1.117175e+04   
50%           3.239000e+03         0.032390       56.000000  2.624100e+04   
75%           8.534250e+03         0.085342      135.750000  6.837900e+04   
max           1.363508e+06        13.635080    25579.000000  1.003911e+07   

       case_fatality_rate  # covid deaths per 100k  
count         2982.000000              2982.000000  
mean             0.018374                 0.002070  
std              0.009878                 0.007781  
min              0.000978                 0.000010  
25%              0.012162                 0.000230  
50%              0.016518                 0.000560  
75%              0.022519                 0.001358  
max              0.142857                 0.255790  

[8 rows x 21 columns]

Using the information reported from `.describe()`, we can imagine dividing our DataFrame into 4 separate bins, based on the distribution for any given feature. Specifically, based on a particular feature:
- the $1^{st}$ bin will be the data that has values between the **min** and **25%**
- the $2^{nd}$ bin will be the data that has values between **25%** and **50%**
- the $3^{rd}$ bin will be the data that has values between **50%** and **75%**
- the $4^{th}$ bin will be the data that has values between **75%** and **max**

<div class='exercise'><b>Exercise 4.4 [3 pts]: Partitioning our data</b>
    
Complete the `partition_df()` function, which takes as input:
- DataFrame to work with
- feature (e.g., obesity) to filter by
- minimum value
- maximum value

and outputs:
- a subset of the DataFrame that has values between the passed-in minimum and maximum values (inclusively) for the passed-in feature.

For example, if we called `partition_df(merged, 'obesity', 30, 45)`, it should return a subset of the `merged` DataFrame that has obesity values between 30 and 45 (and including the boundary values of 30 and 45).
</div>

In [43]:
def partition_df(df, column_name, minv, maxv):
    # YOUR CODE HERE
    return df[(df[column_name] >= minv) & (df[column_name] <= maxv)]
    # END OF YOUR CODE HERE

<div class='exercise'><b>Exercise 4.5: [4 pts] Exploratory Data Analysis</b>
    
Identify a few features that you're interested in, and inspect if there's any correlation with the COVID data. Specifically, simply run your `partition_df()` function below, many times, each with a different subset of the data -- select a range of values and a particular feature. For example, if I'm interested in __cancer__, I could look at the 4 quartiles (per `.describe()`) and use those ranges of values as I repeatedly execute `partition_df()`. For this exercise, after running the function several times, **write 3-5 sentences about any patterns or correlations you noticed or didn't notice but expected to find.**
</div>

In [44]:
# YOUR CODE HERE
partition_df(merged, 'income', merged['income'].min(), merged['income'].max())[["income","case_fatality_rate", "# covid deaths", "# case per 100k"]].describe()
# 0.018416
partition_df(merged, 'income', merged['income'].min(), 38895)[["income","case_fatality_rate", "# covid deaths", "# case per 100k"]].describe()
# 0.022442
partition_df(merged, 'income', 38895, 45192)[["income","case_fatality_rate", "# covid deaths", "# case per 100k"]].describe()
# 0.019062
partition_df(merged, 'income', 45192, 52526)[["income","case_fatality_rate", "# covid deaths", "# case per 100k"]].describe()
# 0.017060
partition_df(merged, 'income', 52526, merged['income'].max())[["income","case_fatality_rate", "# covid deaths", "# case per 100k"]].describe()
# 0.015099
# END OF YOUR CODE HERE


income  case_fatality_rate  # covid deaths  # case per 100k
count     746.000000          746.000000      746.000000       746.000000
mean    62801.116622            0.015083      397.898123         0.255425
std     10840.465675            0.007441     1320.603668         0.724385
min     52526.000000            0.001283        1.000000         0.000290
25%     55148.500000            0.010172       26.000000         0.019627
50%     59225.000000            0.013889       90.500000         0.067595
75%     66923.750000            0.018671      289.750000         0.231703
max    122641.000000            0.057143    25579.000000        13.635080

<div style='background-color:#F6FEFA;padding:15px'>

**I expect county with high average income would have low case fatality rate. After runing for several times and check the mean value of case fatality rate, I find that county with higher average income has lower case fatality rate on average. One guess is that people living in the counties with higher average income probably have better access to healthcare system, thus lower rate of case fatality.**

</div>


`.describe()` provides these nice summary statistics over any portion of data that we give it. Instead of iteratively inspecting several subsets of the data, let's actually split our DataFrame into new categories; instead of representing all features by floating point numbers, let's create new _categorical_ names for feature(s) based on their numbers. The code below does just this. It creates a new column, `income group` that has 4 possible values, each one corresponding to a quartile of the original `income` values. 

Run the cell below.

In [45]:
bins = [0, 38000, 45000, 52000, 200000]
names = ['income-group-1', 'income-group-2', 'income-group-3', 'income-group-4']
d = dict(enumerate(names, 1))
merged['income group'] = np.vectorize(d.get)(np.digitize(merged['income'], bins))
merged


state             county  hispanic  minority     female  unemployed  \
0     Alabama     Autauga County       2.7      24.6  51.124718         5.2   
1     Alabama     Baldwin County       4.4      16.9  51.058714         5.5   
2     Alabama     Barbour County       4.2      54.3  46.658524         8.9   
3     Alabama        Bibb County       2.4      25.4  45.744021         6.6   
4     Alabama      Blount County       9.0      12.6  50.595661         5.4   
...       ...                ...       ...       ...        ...         ...   
3011  Wyoming  Sweetwater County      16.0      20.4  48.388618         4.6   
3012  Wyoming       Teton County      15.0      18.5  46.913580         3.8   
3013  Wyoming       Uinta County       9.1      12.3  48.969072         4.9   
3014  Wyoming    Washakie County      14.2      17.8  50.102993         4.0   
3015  Wyoming      Weston County       1.4       8.4  47.224954         3.3   

      income  nodegree  bachelor  inactivity  ...  voter_gap  trump  biden  \
0      54366      13.8      21.9        28.6  ...       44.5   71.5   27.0   
1      49626      11.0      28.6        22.3  ...       53.9   76.2   22.3   
2      34971      25.4      13.6        31.8  ...        8.0   53.6   45.6   
3      39546      22.1      10.2        33.9  ...       57.7   78.4   20.7   
4      45567      21.9      12.3        28.0  ...       80.0   89.6    9.6   
...      ...       ...       ...         ...  ...        ...    ...    ...   
3011   72604       9.5      18.1        26.0  ...       50.7   73.7   23.0   
3012   75348       4.3      51.9        10.8  ...      -37.5   29.6   67.1   
3013   56800      10.8      18.7        21.5  ...       62.8   79.7   16.9   
3014   50802      10.9      21.2        24.4  ...       64.7   80.9   16.2   
3015   55520      11.4      16.8        22.6  ...       77.5   87.7   10.2   

      # total covid cases  # case per 100k  # covid deaths  population  \
0                    9207          0.09207             119     55869.0   
1                   34726          0.34726             411    223234.0   
2                    3260          0.03260              67     24686.0   
3                    3752          0.03752              74     22394.0   
4                    9095          0.09095             146     57826.0   
...                   ...              ...             ...         ...   
3011                 6111          0.06111              51     42343.0   
3012                 4611          0.04611              11     23464.0   
3013                 3204          0.03204              16     20226.0   
3014                 1120          0.01120              27      7805.0   
3015                  829          0.00829               6      6927.0   

      case_fatality_rate  # covid deaths per 100k    income group  
0               0.012925                  0.00119  income-group-4  
1               0.011836                  0.00411  income-group-3  
2               0.020552                  0.00067  income-group-1  
3               0.019723                  0.00074  income-group-2  
4               0.016053                  0.00146  income-group-3  
...                  ...                      ...             ...  
3011            0.008346                  0.00051  income-group-4  
3012            0.002386                  0.00011  income-group-4  
3013            0.004994                  0.00016  income-group-4  
3014            0.024107                  0.00027  income-group-3  
3015            0.007238                  0.00006  income-group-4  

[2982 rows x 24 columns]

<div class='exercise'><b>Exercise 4.6 [5 pts]: Aggregate data</b>
    
    
Write code in the cell below to group (and display) the data according to the 4 income groups. Also, while we will still keep the same columns (i.e, features), the values of each should now represent the __average__ value of all rows that were subsumed in the making of the aggregate income-group. Your resulting DataFrame should have just 4 rows (income-group-1, income-group-2, income-group-3, income-group-4). See example in the cell below.


Since every feature (except for `# total cases`, `# covid deaths`, and `population`) was already an average value corresponding to a particular __county__, when we aggregate our data by income groups, we are effectively taking an average of an average. Many counties are being aggregated for each income-group row. This approach isn't as accurate as possible; it would be more accurate if we re-adjusted every value so that it was truly an average that was based on the total __population__ of all counties that are subsumed within a given income-group row. That's okay, though. An average of averages will suffice for the purpose of this exercise. 
</div>

In [46]:
# EXAMPLE: If our `merged` DataFrame were
# COUNTY    INCOME GROUP    BACHELOR ... (other columns, too)
#   A            2             50
#   B            1             20
#   C            1             30
#   D            2             70
#   E            3             95

# it should become
# INCOME GROUP    BACHELOR ... (other columns, too)
#   1                25
#   2                60
#   3                95

# YOUR CODE HERE
merged.groupby("income group").agg("mean")
# END OF YOUR CODE HERE

hispanic   minority     female  unemployed        income  \
income group                                                                
income-group-1   9.128049  32.437348  49.996752    7.288720  33925.432927   
income-group-2   8.869704  20.386946  49.813455    5.718696  41627.992620   
income-group-3   8.860669  18.001534  49.863219    4.852301  48492.580195   
income-group-4  10.145352  20.698116  50.008563    4.361809  62139.851759   

                 nodegree   bachelor  inactivity    obesity     density  ...  \
income group                                                             ...   
income-group-1  21.985823  13.590091   30.095427  33.639177   87.712195  ...   
income-group-2  15.617712  17.565068   26.996187  31.489914  131.076630  ...   
income-group-3  12.760530  20.438773   25.085914  30.619386  341.441562  ...   
income-group-4  10.541206  27.359296   22.296608  28.636809  348.928392  ...   

                voter_turnout  voter_gap      trump      biden  \
income group                                                     
income-group-1      41.844069  33.203988  65.926074  32.722086   
income-group-2      37.075902  40.729543  69.545117  28.815575   
income-group-3      34.282463  35.691831  66.912254  31.220423   
income-group-4      29.680387  22.443742  60.226839  37.783097   

                # total covid cases  # case per 100k  # covid deaths  \
income group                                                           
income-group-1          3901.925305         0.039019       84.568598   
income-group-2          8574.188192         0.085742      143.553506   
income-group-3         12825.659693         0.128257      192.301255   
income-group-4         24855.223618         0.248552      386.125628   

                   population  case_fatality_rate  # covid deaths per 100k  
income group                                                                
income-group-1   27788.071646            0.022773                 0.000846  
income-group-2   64545.897909            0.019116                 0.001436  
income-group-3   98807.927476            0.017058                 0.001923  
income-group-4  219901.216080            0.015177                 0.003861  

[4 rows x 21 columns]

<div class='exercise'><b>Wrapping Up</b>
</div>

<div class='exercise'><b>Exercise 4.7.1 [1 pt]: Conclusions</b>
What are your conclusions/finding from this alternative view of the data? (2-4 sentences).
</div>

<div style='background-color:#F6FEFA;padding:15px'>

**Counties with higher average income have lower case fatality rate. Since the higher income the larger population, it may be not meaningful to look into the number of total covid cases, cases per 100k, covid deaths, or covid deaths per 100k to find any pattern or correlation.**

</div>


<div class='exercise'><b>Exercise 4.7.2 [1 pt]: Possible Weaknesses</b>
What are some weaknesses from this view of the data? (2-4 sentences).
</div>

<div style='background-color:#F6FEFA;padding:15px'>

**There might be confounders in the data. If we don't control for the potential confounders (e.g. a common cause of high income and effectiveness of COVID19 testing), there might be confounding that leads to biased estimation of the assocition between income and the effectiveness and thoroughness of COVID19 testing (case fatality rate).**

</div>


## Moving Forward

In this homework assignment, we've focused on gathering, parsing, and exploring data. However, what if we wanted to *predict* some behavior of the data. For example, imagine one is curious how a particular county will respond to COVID. Or, imagine we looked at counties' COVID data on a weekly basis, one could be interested in predicting the upcoming week's behavior.

Alternatively, one could be interested in *inference*, whereby we are more concerned with trying to understand __why__ and __how__ a system behaves the way it does. We might wish to understand which factors most correlate and cause a certain event to happen. This could give us insights into where certain inequalities persist.

For both *prediction* and *inference*, our computational method of solving such a task is referred to as a model. For the remainder of CS109, we will spend significant focus on various models.
</div>

## Reflection

As a reminder, this is just **one** of the homework assignments in this course, the point of which is to assess your learning and to provide both you and us with an indication as to how aligned your knowledge and skills are with our learning objectives. To this end, we encourage you to reflect on your progress, strengths, and weaknesses and to make changes, if necessary, to accomplish your goals. Likewise, please reach out to the TFs and teaching staff if you need help. We want everyone to feel comfortable in being honest about these elements, with both herself/himself and us. For these purposes, we will ask you several times throughout the semester to complete an anonymous poll.